In [2]:
from model import network_mnist,naive_train,test_taskwise,test,benchmark,train_stream,test_stream
from torch.utils.data import DataLoader, Subset
import torch
import torch.nn as nn
import torch.optim as optim


/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

model=network_mnist(256,128)
#print(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs=5

acc_lists=[]
for task in range(5):
    print(f"Training on task {task}...")
    naive_train(model,task, epochs,criterion=criterion,optimizer=optimizer,device=device)
    print(f"Testing on task {task}...")
    #test_taskwise(model,task,device)
    acc,acc_list=test(model,device)
    acc_lists.append(acc_list)



Training on task 0...
Testing on task 0...
Accuracy on task 0: 99.91%
Accuracy on task 1: 0.00%
Accuracy on task 2: 0.00%
Accuracy on task 3: 0.00%
Accuracy on task 4: 0.00%
Average Accuracy: 19.98%
Training on task 1...
Testing on task 1...
Accuracy on task 0: 0.00%
Accuracy on task 1: 99.56%
Accuracy on task 2: 0.00%
Accuracy on task 3: 0.00%
Accuracy on task 4: 0.00%
Average Accuracy: 19.91%
Training on task 2...
Testing on task 2...
Accuracy on task 0: 0.00%
Accuracy on task 1: 0.00%
Accuracy on task 2: 99.68%
Accuracy on task 3: 0.00%
Accuracy on task 4: 0.00%
Average Accuracy: 19.94%
Training on task 3...
Testing on task 3...
Accuracy on task 0: 0.00%
Accuracy on task 1: 0.00%
Accuracy on task 2: 0.00%
Accuracy on task 3: 99.75%
Accuracy on task 4: 0.00%
Average Accuracy: 19.95%
Training on task 4...
Testing on task 4...
Accuracy on task 0: 0.00%
Accuracy on task 1: 0.00%
Accuracy on task 2: 0.00%
Accuracy on task 3: 0.00%
Accuracy on task 4: 98.79%
Average Accuracy: 19.76%
